In [ ]:
#import library yang dibutuhkan
#pandas digunakan untuk membuka file berbentuk excel yang berisi data-data yang dibutuhkan
import pandas as pd
#random digunakan untuk mendapatkan nilai random untuk initial solution
import random as rd
#digunakan untuk melakukan kombinasi
from itertools import combinations
#untuk kalkulasi matematika
import math

#membuat sebuah class untuk tabu search
class TS():
    def __init__(self, Path, seed, tabu_tenure):
        #letak dari dataset yang akan digunakan
        self.Path = Path
        #untuk melakukan random initial solution
        self.seed = seed
        #tabu tenure digunakan untuk berapa banyak komponen yang bisa dianggap tabu
        self.tabu_tenure = tabu_tenure
        #menyimpan data dalam variabel instance_dict
        self.instance_dict = self.input_data()
        #melakukan inisiasi solusi awal secara random
        self.Initial_solution = self.get_InitialSolution()
        #membuat fungsi TSearch yang berisi nilai tabu, nilai solusi terbaik, dan nilai objective function terbaik
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()

    def input_data(self):
        #mengambil data dalam bentuk excel dan mengembalikannya dalam bentuk dict, 
        #dimana job akan bertindak sebagai index, weight, processing_time, dan due_date
        return pd.read_excel(self.Path, names=['Job', 'weight', "processing_time", "due_date"],index_col=0).to_dict('index')

    def get_tabuestructure(self):
        #mengambil dict dari fungsi input_data
        #mengembalikan dict masing-masing job yang telah dilakukan kombinasi dengan tabu_time dan MoreValue
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict

    def get_InitialSolution(self, show=False):
        #mendapatkan jumlah job
        n_jobs = len(self.instance_dict) 
        #memproduksi nilai random dari job
        initial_solution = list(range(1, n_jobs+1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        if show == True:
            print("Initial random solution adalah: {}".format(initial_solution))
        return initial_solution

    def Objfun(self, solution, show = False):
        #mendapatkan job dari dataset dan mengembalikan nilai objective function
        dict = self.instance_dict
        #start time adalah 0
        t = 0  
        #nilai objective value dimulai dari 0
        objfun_value = 0
        #untuk setiap solusi akan dihitup nilai objective value-nya
        for job in solution:
            #waktu proses untuk selesai
            C_i = t + dict[job]["processing_time"]
            #due date untuk job
            d_i = dict[job]["due_date"]
            #waktu keterlambatan
            T_i = max(0, C_i - d_i)
            #weight job
            W_i = dict[job]["weight"]
            #nilai objective function didapatkan dengan melakukan perkalian weight dengan nilai keterlambatan
            objfun_value +=  W_i * T_i
            #menyimpan nilai waktu proses untuk selesai di variabel t
            t = C_i
        if show == True:
            print("The Objective function value for {} solution schedule is: {}".format(solution ,objfun_value))
        return objfun_value

    def SwapMove(self, solution, i ,j):
        #mengambil nilai solution dalam bentuk list dan melakukan swap i dan j
        solution = solution.copy()
        #mencari index i dan j
        i_index = solution.index(i)
        j_index = solution.index(j)
        #melakukan swap
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution

    def TSearch(self):
        #implementasi algoritma Tabu Search 
        #parameter tenure
        tenure =self.tabu_tenure
        #struktur dari data
        tabu_structure = self.get_tabuestructure()  
        #solusi terbaik
        best_solution = self.Initial_solution
        #objective value terbaik
        best_objvalue = self.Objfun(best_solution)
        #solusi sekarang
        current_solution = self.Initial_solution
        #objective value sekarang
        current_objvalue = self.Objfun(current_solution)
        print()
        print("Short-term memory TS with Tabu Tenure: {}\nInitial Solution: {}, Initial Objvalue: {}".format(
            tenure, current_solution, current_objvalue))
        iter = 1
        Terminate = 0
        while Terminate < 100:
            print()
            print('Iter {} Current_Objvalue: {}, Best_Objvalue: {}'.format(iter, current_objvalue, best_objvalue))
            #mencari seluruh neighborhood pada solusi
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue

            #langkah yang bisa dilakukan
            while True:
                #memilih objective value yang terendah (minimasi)
                best_move = min(tabu_structure, key =lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]["MoveValue"]
                tabu_time = tabu_structure[best_move]["tabu_time"]
                #jika tidak tabu
                if tabu_time < iter:
                    #melakukan swap
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    #mengganti nilai best dengan current
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("best_move: {}, Objvalue: {} => Best Improving => Admissible".format(best_move,current_objvalue))
                        Terminate = 0
                    else:
                        print("Termination: {}  best_move: {}, Objvalue: {} => Least non-improving => ""Admissible".format(Terminate,best_move,current_objvalue))
                        Terminate += 1
                    #melakukan update tabu_time 
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                #Jika tabu
                else:
                    #nilai aspirasi
                    if MoveValue < best_objvalue:
                        #melakukan swap dan mengganti nilai best dengan current
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("best_move: {}, Objvalue: {} => Aspiration => Admissible".format(best_move,current_objvalue))
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]["MoveValue"] = float('inf')
                        print("best_move: {}, Objvalue: {} => Tabu => Inadmissible".format(best_move, current_objvalue))
                        continue
        print("Performed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,best_objvalue), sep="\n")
        return tabu_structure, best_solution, best_objvalue


test = TS(Path="/content/Data Dummy OKH.xlsx", seed = 2012, tabu_tenure=3)


Short-term memory TS with Tabu Tenure: 3
Initial Solution: [4, 6, 5, 1, 7, 9, 3, 8, 2], Initial Objvalue: 4271

Iter 1 Current_Objvalue: 4271, Best_Objvalue: 4271
best_move: (2, 4), Objvalue: 3434 => Best Improving => Admissible

Iter 2 Current_Objvalue: 3434, Best_Objvalue: 3434
best_move: (7, 9), Objvalue: 3275 => Best Improving => Admissible

Iter 3 Current_Objvalue: 3275, Best_Objvalue: 3275
best_move: (4, 8), Objvalue: 3228 => Best Improving => Admissible

Iter 4 Current_Objvalue: 3228, Best_Objvalue: 3228
Termination: 0  best_move: (1, 5), Objvalue: 3228 => Least non-improving => Admissible

Iter 5 Current_Objvalue: 3228, Best_Objvalue: 3228
Termination: 1  best_move: (1, 2), Objvalue: 3228 => Least non-improving => Admissible

Iter 6 Current_Objvalue: 3228, Best_Objvalue: 3228
best_move: (1, 2), Objvalue: 3228 => Tabu => Inadmissible
best_move: (1, 5), Objvalue: 3228 => Tabu => Inadmissible
Termination: 2  best_move: (1, 6), Objvalue: 3228 => Least non-improving => Admissible

